In [ ]:
latest_gameweek = 20

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import catboost
import shap

import plotly.graph_objects as go
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 300)
#pd.set_option('display.max_rows', 100)

# Data

In [ ]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
display(projections.head())
display(projections.shape)

In [ ]:
filepath = Path('../data/fpl_df.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='23-24']
display(fpl_df.head())
display(fpl_df.shape)

In [ ]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 
                            'gameweek_minutes_ewm_20', 'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10', 'gameweek_xPoints_ewm_20', 'gameweek_xPoints_ewm_40']]
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
df['price'] = df['now_cost'] / 10.0
df['value'] = df['gameweek_xPoints_ewm_20'] / df['price']
df['value_points'] = np.sqrt( df['gameweek_xPoints_ewm_20'] *  df['value'])
df

In [ ]:
expected_points_next_10gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+11, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_10_GW'})

expected_points_next_5gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+6, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_5_GW'})

expected_points_next_10gw

In [ ]:
df = df.merge(expected_points_next_10gw, on='name', how='left')
df = df.merge(expected_points_next_5gw, on='name', how='left')
display(df.head())

# Analysis

In [ ]:
(df[(df.games_played>=5)]
        .sort_values(by='expected_points_next_10_GW', ascending=False)
        .head(20))[['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_20', 'expected_points_next_10_GW',
                    'value_points']]

In [ ]:
df[df['name'].str.contains('Pickford')][['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10',
                    'gameweek_xPoints_ewm_20', 'expected_points_next_10_GW', 'value_points']]

In [ ]:
positions = ['GOALKEEPERS', 'DEFENDERS', 'MIDFIELDERS', 'FORWARDS']

for ix, position in enumerate(positions):
    print(position)
    display(
        (df[(df.games_played>=5) & (df.element_type==ix+1)]
        .sort_values(by='expected_points_next_10_GW', ascending=False) 
        .head(20))[['name', 'team_name', 'price', 'games_played',
                    'points_per_game', 'total_points', 'gameweek_xPoints_ewm_20',
                    'expected_points_next_10_GW', 'value_points']]
    )


In [ ]:
my_price = 6.1
my_player = 'Gordon'
print('Value_points with my_price:')
print(np.sqrt(df.loc[df['name'].str.contains(my_player), 'gameweek_xPoints_ewm_20']**2/my_price))

In [ ]:
fpl_df.loc[fpl_df['name'].str.contains('Gordon'), ['name', 'event_points', 'gameweek_xPoints', 'gameweek_xPoints_expanding_per90']] #André Onana

In [ ]:
(projections[projections.gameweek==(latest_gameweek+1)]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

In [ ]:
(projections[(projections.gameweek.isin(np.arange(latest_gameweek+1, latest_gameweek+11)))]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['expected_points']]

In [ ]:
projections[projections['name'].str.contains('Onana')].name.unique()

In [ ]:
fpl_df.loc[fpl_df['name'].str.contains('Areola'), ['name', 'gameweek', 'gameweek_minutes', 'gameweek_xPoints', ]]

In [ ]:
my_gameweek = latest_gameweek+1

my_starting_11_names = ['Pickford', 'Trippier', 'Porro', 'Estupi', 'Saka', 'Bowen', 'Foden', 'Palmer', 'Julián Álvarez',  
                  'Haaland', 'Watkins']
my_subs_names = ['Turner', 'Gordon', 'Konsa', 'Taylor', ]

my_starting_11 = pd.DataFrame()
for my_name in my_starting_11_names:
    my_starting_11 = pd.concat([
                    my_starting_11, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

my_subs = pd.DataFrame()
for my_name in my_subs_names:
    my_subs = pd.concat([
                    my_subs, 
                    projections[(projections.gameweek==my_gameweek) & (projections.name.str.contains(my_name))]
                    ])

expected_points = my_starting_11['expected_points'].sum() + my_starting_11['expected_points'].max()
display(f'Expected points: {expected_points}')

display(my_starting_11[['name', 'element_type', 'home', 'expected_points']])
display(my_starting_11.shape[0])

display(my_subs[['name', 'element_type', 'home', 'expected_points']])
display(my_subs.shape[0])

In [ ]:
(projections[(projections.gameweek==(latest_gameweek+1))]
 .groupby('name')
 .sum()
 .sort_values('expected_points', ascending=False).head(20))[['now_cost', 'expected_points']]

In [ ]:
players = ['Pervis Estupiñán', 'Gabriel dos Santos Magalhães']
colors = ['red', 'white']
#players = ['Heung-Min Son', 'Bruno Borges Fernandes']
#colors = ['white', 'red']

fig = go.Figure()
for count, player in enumerate(players):

    my_projections = projections[projections['name']==player].sort_values(by='gameweek')

    fig.add_trace(
        go.Scatter(
            x=np.unique(my_projections['gameweek']),
            y=my_projections.groupby('gameweek').sum()['expected_points'],#.cumsum(),
            mode="markers+lines",
            marker=dict(color=colors[count]),
            name=player + ' - projected',
            #hovertext=('Opponent: ' + my_projections['opp_team'] + 
            #           ', was_home: ' + my_projections['was_home'].astype(str)),
            showlegend=True,
            ),
    )

fig.update_layout(
    #title="",
    template='plotly_dark',
    xaxis_title="gameweek",
    yaxis_title='projected_points',
    #showlegend=True
)

In [ ]:
for player in players:
    my_projections = projections[projections['name']==player].sort_values(by='gameweek')
    print(player)
    display(my_projections[['opponent_team','home','gameweek', 'expected_points']].set_index('gameweek'))

## SHAP

In [ ]:
model_path = Path('../models/catboost_20240102-210118.cbm')
model = catboost.CatBoostRegressor()
model.load_model(model_path)

In [ ]:
features_no_shift = ['element_type', 'home', 'opponent_xG_ewm_5', 'opponent_xG_ewm_10',
       'opponent_xG_ewm_20', 'opponent_xG_ewm_40', 'opponent_xGA_ewm_5',
       'opponent_xGA_ewm_10', 'opponent_xGA_ewm_20',
       'opponent_xGA_ewm_40', ]

features_shift = ['corners_and_indirect_freekicks_order', 'creativity_rank', 
       'direct_freekicks_order', 'ict_index_rank', 'influence_rank',
       'minutes', 'now_cost', 'penalties_order', 'points_per_game', 
       'selected_by_percent', 'threat_rank',
       'team_xG_ewm_5', 'team_xG_ewm_10', 'team_xG_ewm_20',
       'team_xG_ewm_40', 'team_xGA_ewm_5', 'team_xGA_ewm_10',
       'team_xGA_ewm_20', 'team_xGA_ewm_40', 
       'gameweek_assists_ewm_5', 'gameweek_bps_ewm_5',
       'gameweek_creativity_ewm_5', 'event_points_ewm_5',
       'gameweek_goals_scored_ewm_5', 'gameweek_goals_conceded_ewm_5',
       'gameweek_saves_ewm_5', 'gameweek_threat_ewm_5',
       'gameweek_xG_ewm_5', 'gameweek_xA_ewm_5', 'gameweek_xGA_ewm_5',
       'gameweek_minutes_ewm_5', 'gameweek_xPoints_ewm_5',
       'gameweek_assists_ewm_10', 'gameweek_bps_ewm_10',
       'gameweek_creativity_ewm_10', 'event_points_ewm_10',
       'gameweek_goals_scored_ewm_10', 'gameweek_goals_conceded_ewm_10',
       'gameweek_saves_ewm_10', 'gameweek_threat_ewm_10',
       'gameweek_xG_ewm_10', 'gameweek_xA_ewm_10', 'gameweek_xGA_ewm_10',
       'gameweek_minutes_ewm_10', 'gameweek_xPoints_ewm_10',
       'gameweek_assists_ewm_20', 'gameweek_bps_ewm_20',
       'gameweek_creativity_ewm_20', 'event_points_ewm_20',
       'gameweek_goals_scored_ewm_20', 'gameweek_goals_conceded_ewm_20',
       'gameweek_saves_ewm_20', 'gameweek_threat_ewm_20',
       'gameweek_xG_ewm_20', 'gameweek_xA_ewm_20', 'gameweek_xGA_ewm_20',
       'gameweek_minutes_ewm_20', 'gameweek_xPoints_ewm_20',
       'gameweek_assists_ewm_40', 'gameweek_bps_ewm_40',
       'gameweek_creativity_ewm_40', 'event_points_ewm_40',
       'gameweek_goals_scored_ewm_40', 'gameweek_goals_conceded_ewm_40',
       'gameweek_saves_ewm_40', 'gameweek_threat_ewm_40',
       'gameweek_xG_ewm_40', 'gameweek_xA_ewm_40', 'gameweek_xGA_ewm_40',
       'gameweek_minutes_ewm_40', 'gameweek_xPoints_ewm_40',
       'gameweek_assists_expanding', 'gameweek_bps_expanding',
       'gameweek_creativity_expanding', 'event_points_expanding',
       'gameweek_goals_scored_expanding',
       'gameweek_goals_conceded_expanding', 'gameweek_saves_expanding',
       'gameweek_threat_expanding', 'gameweek_xG_expanding',
       'gameweek_xA_expanding', 'gameweek_xGA_expanding',
       'gameweek_minutes_expanding', 'gameweek_xPoints_expanding',
       'gameweek_assists_expanding_per90', 'gameweek_bps_expanding_per90',
       'gameweek_creativity_expanding_per90',
       'event_points_expanding_per90',
       'gameweek_goals_scored_expanding_per90',
       'gameweek_goals_conceded_expanding_per90',
       'gameweek_saves_expanding_per90',
       'gameweek_threat_expanding_per90', 'gameweek_xG_expanding_per90',
       'gameweek_xA_expanding_per90', 'gameweek_xGA_expanding_per90',
       'gameweek_xPoints_expanding_per90', 'xG_overperformance'
    ]

features = features_no_shift + features_shift

target = ['event_points']

In [ ]:
my_player = 'Konsa'
projections.loc[projections['name'].str.contains(my_player), ['name', 'team_name', 'home', 'expected_points']]

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(projections.loc[projections['name'].str.contains(my_player), features]);

In [ ]:
shap.plots.bar(shap_values, max_display=20)